In [13]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import newton
import matplotlib.pyplot as plt

Сорри, питон плохо качал таблицы из экселя, мне уже легче от руки перебить

In [14]:
data_rates = {
    'Risk-free rate': [0.0409, 0.0412, 0.0414, 0.0417, 0.0419, 0.0422, 0.0424, 0.0427, 0.0429, 0.0432],
    'Maturity': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'AAA': [0.0004, 0.0010, 0.0013, 0.0017, 0.0018, 0.0020, 0.0021, 0.0026, 0.0028, 0.0031],
    'AA': [0.0014, 0.0025, 0.0029, 0.0033, 0.0034, 0.0035, 0.0037, 0.0042, 0.0044, 0.0046],
    'A': [0.0024, 0.0035, 0.0040, 0.0046, 0.0048, 0.0051, 0.0053, 0.0060, 0.0066, 0.0066],
    'BBB': [0.0034, 0.0051, 0.0061, 0.0071, 0.0076, 0.0082, 0.0087, 0.0096, 0.0105, 0.0107],
    'BB': [0.0109, 0.0140, 0.0158, 0.0175, 0.0186, 0.0184, 0.0182, 0.0185, 0.0189, 0.0191],
    'B': [0.0194, 0.0230, 0.0251, 0.0268, 0.0279, 0.0277, 0.0276, 0.0278, 0.0283, 0.0284],
    'CCC': [0.0439, 0.0495, 0.0508, 0.0519, 0.0525, 0.0523, 0.0521, 0.0524, 0.0528, 0.0530]
}

df_rates = pd.DataFrame(data_rates)
df_rates

,Risk-free rate,Maturity,AAA,AA,A,BBB,BB,B,CCC
0,0.0409,1,0.0004,0.0014,0.0024,0.0034,0.0109,0.0194,0.0439
1,0.0412,2,0.0010,0.0025,0.0035,0.0051,0.0140,0.0230,0.0495
2,0.0414,3,0.0013,0.0029,0.0040,0.0061,0.0158,0.0251,0.0508
3,0.0417,4,0.0017,0.0033,0.0046,0.0071,0.0175,0.0268,0.0519
4,0.0419,5,0.0018,0.0034,0.0048,0.0076,0.0186,0.0279,0.0525
5,0.0422,6,0.0020,0.0035,0.0051,0.0082,0.0184,0.0277,0.0523
6,0.0424,7,0.0021,0.0037,0.0053,0.0087,0.0182,0.0276,0.0521
7,0.0427,8,0.0026,0.0042,0.0060,0.0096,0.0185,0.0278,0.0524
8,0.0429,9,0.0028,0.0044,0.0066,0.0105,0.0189,0.0283,0.0528
9,0.0432,10,0.0031,0.0046,0.0066,0.0107,0.0191,0.0284,0.0530


In [15]:
historical_matrix = pd.DataFrame(
    [[0.9081, 0.0833, 0.0068, 0.0006, 0.0012, 0.0000, 0.0000, 0.0000],
     [0.0070, 0.9065, 0.0779, 0.0064, 0.0006, 0.0014, 0.0002, 0.0000],
     [0.0009, 0.0227, 0.9105, 0.0552, 0.0074, 0.0026, 0.0001, 0.0006],
     [0.0002, 0.0033, 0.0595, 0.8693, 0.0530, 0.0117, 0.0012, 0.0018],
     [0.0003, 0.0014, 0.0067, 0.0773, 0.8053, 0.0884, 0.0100, 0.0106],
     [0.0000, 0.0011, 0.0024, 0.0043, 0.0648, 0.8347, 0.0407, 0.0520],
     [0.0022, 0.0000, 0.0022, 0.0130, 0.0238, 0.1124, 0.6485, 0.1979],
     [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000]],
    index=['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'D'],
    columns=['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'D']
)

historical_matrix

,AAA,AA,A,BBB,BB,B,CCC,D
AAA,0.9081,0.0833,0.0068,0.0006,0.0012,0.0000,0.0000,0.0000
AA,0.0070,0.9065,0.0779,0.0064,0.0006,0.0014,0.0002,0.0000
A,0.0009,0.0227,0.9105,0.0552,0.0074,0.0026,0.0001,0.0006
BBB,0.0002,0.0033,0.0595,0.8693,0.0530,0.0117,0.0012,0.0018
BB,0.0003,0.0014,0.0067,0.0773,0.8053,0.0884,0.0100,0.0106
B,0.0000,0.0011,0.0024,0.0043,0.0648,0.8347,0.0407,0.0520
CCC,0.0022,0.0000,0.0022,0.0130,0.0238,0.1124,0.6485,0.1979
D,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000


In [16]:
edf_matrix = pd.DataFrame(
    [[0.6626, 0.2222, 0.0737, 0.0245, 0.0086, 0.0067, 0.0014, 0.0002],
     [0.2166, 0.4304, 0.2583, 0.0656, 0.0199, 0.0068, 0.0020, 0.0004],
     [0.0276, 0.2034, 0.4419, 0.2294, 0.0742, 0.0197, 0.0028, 0.0010],
     [0.0030, 0.0280, 0.2263, 0.4254, 0.2352, 0.0695, 0.0100, 0.0026],
     [0.0008, 0.0024, 0.0369, 0.2293, 0.4441, 0.2453, 0.0341, 0.0071],
     [0.0001, 0.0005, 0.0039, 0.0348, 0.2047, 0.5300, 0.2058, 0.0201],
     [0.0000, 0.0001, 0.0009, 0.0026, 0.0179, 0.1777, 0.6994, 0.1013],
     [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000]],
    index=['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'D'],
    columns=['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'D']
)

edf_matrix

,AAA,AA,A,BBB,BB,B,CCC,D
AAA,0.6626,0.2222,0.0737,0.0245,0.0086,0.0067,0.0014,0.0002
AA,0.2166,0.4304,0.2583,0.0656,0.0199,0.0068,0.0020,0.0004
A,0.0276,0.2034,0.4419,0.2294,0.0742,0.0197,0.0028,0.0010
BBB,0.0030,0.0280,0.2263,0.4254,0.2352,0.0695,0.0100,0.0026
BB,0.0008,0.0024,0.0369,0.2293,0.4441,0.2453,0.0341,0.0071
B,0.0001,0.0005,0.0039,0.0348,0.2047,0.5300,0.2058,0.0201
CCC,0.0000,0.0001,0.0009,0.0026,0.0179,0.1777,0.6994,0.1013
D,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000


In [17]:
my_rating = 'AAA'
my_maturity = 6
my_coupon_rate = 0.06
my_face_value = 1000
my_RR = 0.538

### 1. Compute the current price and credit spread

In [18]:
current_price = 0
for year in range(1, my_maturity + 1):
    risk_free = df_rates.loc[df_rates['Maturity'] == year, 'Risk-free rate'].values[0]
    spread = df_rates.loc[df_rates['Maturity'] == year, my_rating].values[0]
    discount_rate = risk_free + spread # находим ставку как риск-фри + спред для каждого срока
    
    if year < my_maturity:
        cash_flow = my_face_value * my_coupon_rate #купон
    else:
        cash_flow = my_face_value * (1 + my_coupon_rate) # купон + номинал в последний год
    
    current_price += cash_flow / ((1 + discount_rate) ** year)

print(f"Текущая цена облигации: {current_price:.2f}")

Текущая цена облигации: 1082.58


Получается, что облигация торгуется с премией.

In [19]:
# функция для YTM, аналог солвера в экселе

# уравнение, которое приравнивает цену облигации и денежные потоки по ней
def ytm_equation(ytm, price, maturity, coupon, face):
    pv = 0
    for t in range(1, maturity+1): # проходим по всем годам до погашения облигации
        if t < maturity:
            cf = coupon * face
        else:
            cf = (coupon * face) + face # в последний год + номинал
        pv += cf / ((1 + ytm)**t) 
    return pv - price

ytm = newton(ytm_equation, x0=0.05, args=(current_price, my_maturity, my_coupon_rate, my_face_value))
print(f"YTM: {ytm*100:.2f}%")

risk_free_6y = df_rates.loc[df_rates['Maturity'] == 6, 'Risk-free rate'].values[0]
credit_spread = ytm - risk_free_6y
print(f"Кредитный спред: {credit_spread*100:.2f}%")

YTM: 4.40%
Кредитный спред: 0.18%


### 2. Historical and EDF transition matrix, calculate bond price in 1 year and VaR

#### a) расчёт форвардных ставок

In [20]:
def calculate_forward_rates(df_rates):
    forward_rates = pd.DataFrame()
    
    for rating in df_rates.columns[2:]:  # Все рейтинги кроме 'Maturity' и 'Risk-free rate'
        spot_rates = df_rates['Risk-free rate'] + df_rates[rating]
        rates = []
        
        for t in range(1, len(spot_rates)):
            numerator = (1 + spot_rates[t])**(t + 1)
            denominator = (1 + spot_rates[0])
            forward_rate = (numerator / denominator)**(1/t) - 1
            rates.append(forward_rate)
        
        forward_rates[rating] = rates
    
    forward_rates['Period'] = [f"1→{i+1} ({i})" for i in range(1, len(forward_rates)+1)]
    return forward_rates.set_index('Period')

forward_rates_df = calculate_forward_rates(df_rates)

forward_rates_df

,AAA,AA,A,BBB,BB,B,CCC
Period,,,,,,,
1→2 (1),0.043101,0.045102,0.046102,0.048304,0.058611,0.068114,0.096632
1→3 (2),0.043401,0.045301,0.046452,0.049104,0.059910,0.069614,0.095919
1→4 (3),0.044101,0.045902,0.047302,0.050304,0.061678,0.071247,0.096549
1→5 (4),0.044301,0.046051,0.047552,0.050804,0.062686,0.072188,0.096813
1→6 (5),0.044781,0.046381,0.048102,0.051624,0.062369,0.071830,0.096450
1→7 (6),0.045034,0.046735,0.048435,0.052238,0.062074,0.071625,0.096125
1→8 (7),0.045873,0.047559,0.049474,0.053448,0.062550,0.071965,0.096579
1→9 (8),0.046251,0.047927,0.050278,0.054543,0.063057,0.072570,0.097070
1→10 (9),0.046857,0.048413,0.050525,0.054972,0.063473,0.072863,0.097474


#### b) расчёт форвардной цены облигации для каждого рейтинга

In [21]:
maturity_left = my_maturity - 1 # смещение на год

def calculate_forward_price(rating, maturity_left, face_value, coupon_rate, recovery_rate, forward_rates_df):
    if rating == 'D': # если дефолт, то денежный поток - recovery rate по сути
        return face_value * recovery_rate
    
    price = 0
    for year in range(1, maturity_left + 1):
        # Получаем форвардную ставку для соответствующего периода
        # Периоды в forward_rates_df: 1→2 (1), 1→3 (2), 1→4 (3), ...
        period_idx = year - 1  # Индекс периода (0-based)
        
        # Если срок больше максимального в таблице, используем последнюю доступную ставку
        if period_idx >= len(forward_rates_df):
            period_idx = len(forward_rates_df) - 1
        
        forward_rate = forward_rates_df[rating].iloc[period_idx]
        
        if year < maturity_left:
            cash_flow = face_value * coupon_rate
        else:
            cash_flow = face_value * (1 + coupon_rate)
        
        price += cash_flow / ((1 + forward_rate) ** year)
    
    return (price + face_value * coupon_rate) # d статье купон первого года прибавляется, но не дисконтируется

# Рассчитываем форвардные цены для всех возможных рейтингов
ratings = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'D']
forward_prices = {}

print("Форвардные цены через 1 год:")
for rating in ratings:
    forward_prices[rating] = calculate_forward_price(
        rating, maturity_left, my_face_value, my_coupon_rate, my_RR, forward_rates_df
    )
    print(f"Рейтинг {rating}: {forward_prices[rating]:.2f}")

Форвардные цены через 1 год:
Рейтинг AAA: 1127.29
Рейтинг AA: 1119.87
Рейтинг A: 1112.29
Рейтинг BBB: 1096.89
Рейтинг BB: 1050.57
Рейтинг B: 1012.16
Рейтинг CCC: 920.53
Рейтинг D: 538.00


#### с) historical transition matrix

In [22]:
historical_expected_price = sum(historical_matrix.loc[my_rating] * [forward_prices[r] for r in historical_matrix.columns])
print(f"Ожидаемая цена через 1 год (historical transition matrix): {historical_expected_price:.2f}")

historical_price_distribution = pd.DataFrame({
    'Рейтинг': historical_matrix.columns,
    'Вероятность': historical_matrix.loc[my_rating],
    'Цена': [forward_prices[r] for r in historical_matrix.columns],
    'Изменение': [forward_prices[r] - historical_expected_price for r in historical_matrix.columns]
}).sort_values('Изменение')

historical_price_distribution['Накопленная вероятность'] = historical_price_distribution['Вероятность'].cumsum()

historical_var_99 = historical_price_distribution[historical_price_distribution['Накопленная вероятность'] >= 0.01]['Изменение'].min()
print(f"99% VaR (историческая матрица): {abs(historical_var_99):.2f}")

# выведем это распределение для наглядности
print("\nРаспределение (историческая матрица):")
print(historical_price_distribution[['Рейтинг', 'Вероятность', 'Цена', 'Изменение', 'Накопленная вероятность']])

Ожидаемая цена через 1 год (historical transition matrix): 1126.46
99% VaR (историческая матрица): 6.58

Распределение (историческая матрица):
    Рейтинг  Вероятность         Цена   Изменение  Накопленная вероятность
D         D       0.0000   538.000000 -588.455989                   0.0000
CCC     CCC       0.0000   920.534342 -205.921647                   0.0000
B         B       0.0000  1012.160212 -114.295776                   0.0000
BB       BB       0.0012  1050.574510  -75.881479                   0.0012
BBB     BBB       0.0006  1096.890127  -29.565862                   0.0018
A         A       0.0068  1112.290104  -14.165885                   0.0086
AA       AA       0.0833  1119.874977   -6.581011                   0.0919
AAA     AAA       0.9081  1127.285549    0.829560                   1.0000


#### d) EDF transition matrix

In [23]:
edf_expected_price = sum(edf_matrix.loc[my_rating] * [forward_prices[r] for r in edf_matrix.columns])
print(f"Ожидаемая цена через 1 год (EDF transition matrix): {edf_expected_price:.2f}")

edf_price_distribution = pd.DataFrame({
    'Рейтинг': edf_matrix.columns,
    'Вероятность': edf_matrix.loc[my_rating],
    'Цена': [forward_prices[r] for r in edf_matrix.columns],
    'Изменение': [forward_prices[r] - edf_expected_price for r in edf_matrix.columns]
}).sort_values('Изменение')

edf_price_distribution['Накопленная вероятность'] = edf_price_distribution['Вероятность'].cumsum()
edf_var_99 = edf_price_distribution[edf_price_distribution['Накопленная вероятность'] >= 0.01]['Изменение'].min()
print(f"99% VaR (EDF матрица): {abs(edf_var_99):.2f}")

print("\nРаспределение (EDF матрица):")
print(edf_price_distribution[['Рейтинг', 'Вероятность', 'Цена', 'Изменение', 'Накопленная вероятность']])

Ожидаемая цена через 1 год (EDF transition matrix): 1121.84
99% VaR (EDF матрица): 71.26

Распределение (EDF матрица):
    Рейтинг  Вероятность         Цена   Изменение  Накопленная вероятность
D         D       0.0002   538.000000 -583.837976                   0.0002
CCC     CCC       0.0014   920.534342 -201.303634                   0.0016
B         B       0.0067  1012.160212 -109.677763                   0.0083
BB       BB       0.0086  1050.574510  -71.263466                   0.0169
BBB     BBB       0.0245  1096.890127  -24.947849                   0.0414
A         A       0.0737  1112.290104   -9.547872                   0.1151
AA       AA       0.2222  1119.874977   -1.962998                   0.3373
AAA     AAA       0.6626  1127.285549    5.447573                   0.9999


VaR edf выше, так как в матрице edf вероятности перейти с ААА на рейтинги ниже повышаются

### 3. Monte Carlo approach

Я моделирую 2 состояния - рецессия и эк. рост, меняю transition matrix

In [ ]:
from tqdm import tqdm

np.random.seed(42) # по классике
n_simulations = 10000
business_cycles = ['recession', 'expansion']
cycle_prob = 0.3  # вероятность рецессии, в классе с Алексеем как раз обсуждали, что сейчас около 30% аналитиков дают прозноз рецессии в США

# корректировка transition matrix для бизнес-цикла
def adjust_matrix(matrix, cycle):
    adjusted = matrix.copy()
    if cycle == 'recession': # усиливаю снижение и дефолт в случае рецессии
        for i in range(len(adjusted)):
            for j in range(i, len(adjusted)):
                if j > i:  # увеличение вероятности понижения рейтинга с коэффициентом, пусть будет, 1,5
                    adjusted.iloc[i,j] *= 1.5
                elif j == len(adjusted)-1:  # увеличение вероятности дефолта
                    adjusted.iloc[i,j] *= 2.0
    else:  # увеличиваю вероятности повышения кредитного рейтинга в случае эк. роста
        for i in range(len(adjusted)):
            for j in range(i + 1):
                if j <= i:  # нам нужна по факту только 1-ая строка таблицы (рейтинг ААА), поэтому повышаем вероятность остаться в нем
                    adjusted.iloc[i,j] *= 1.5
    # нормализую вероятности
    return adjusted.div(adjusted.sum(axis=1), axis=0)

simulated_prices = []
cycle_results = []  # для хранения состояния экономики

for _ in tqdm(range(n_simulations)):
    # определяем состояние экономики
    cycle = np.random.choice(business_cycles, p=[cycle_prob, 1-cycle_prob])
    cycle_results.append(cycle)
    
    current_matrix = adjust_matrix(edf_matrix, cycle)
    # генерируем новый рейтинг
    probs = current_matrix.loc[my_rating]
    new_rating = np.random.choice(probs.index, p=probs)
    
    # расчет цены
    if new_rating == 'D':
        price = my_face_value * my_RR
    else:
        price = my_face_value * my_coupon_rate
        for year in range(1, my_maturity):
            rate = df_rates.loc[df_rates['Maturity'] == year, 'Risk-free rate'].values[0] + \
                   df_rates.loc[df_rates['Maturity'] == year, new_rating].values[0]
            
            if year < my_maturity - 1:
                cf = my_face_value * my_coupon_rate
            else:
                cf = my_face_value * (1 + my_coupon_rate)
            
            price += cf / ((1 + rate) ** year)
    
    simulated_prices.append(price)


simulated_prices = np.array(simulated_prices)
cycle_results = np.array(cycle_results)
price_changes = simulated_prices - np.mean(simulated_prices)

# разделяем результаты по состоянию экономики
recession_prices = simulated_prices[cycle_results == 'recession']
expansion_prices = simulated_prices[cycle_results == 'expansion']

# расчет VaR для каждого состояния экономики
def calculate_var(prices, confidence=0.99):
    changes = prices - np.mean(prices)
    return -np.percentile(changes, 100*(1-confidence))

var_99_recession = calculate_var(recession_prices)
var_99_expansion = calculate_var(expansion_prices)

# VaR без разделения на состояния экономики
var_99_total = calculate_var(simulated_prices)

print(f"Доля рецессий: {len(recession_prices)/n_simulations:.1%}")
print(f"\nОжидаемая цена (общая): {np.mean(simulated_prices):.2f}")
print(f"99% VaR (общий): {var_99_total:.2f}")
print(f"\nОжидаемая цена (рецессия): {np.mean(recession_prices):.2f}")
print(f"99% VaR (рецессия): {var_99_recession:.2f}")
print(f"\nОжидаемая цена (экспансия): {np.mean(expansion_prices):.2f}")
print(f"99% VaR (экспансия): {var_99_expansion:.2f}")

100%|██████████| 10000/10000 [00:20<00:00, 478.04it/s]

Доля рецессий: 30.2%

Ожидаемая цена (общая): 1127.63
99% VaR (общий): 68.02

Ожидаемая цена (рецессия): 1125.69
99% VaR (рецессия): 104.16

Ожидаемая цена (экспансия): 1128.46
99% VaR (экспансия): 68.86
